In [1]:
from pyspark.sql import SparkSession

sp = SparkSession.builder.appName('questions').getOrCreate()
ratings = sp.read.json('movies.json')
ratings.printSchema()

24/10/27 16:22:37 WARN Utils: Your hostname, DESKTOP-2J74AJH resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/27 16:22:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/27 16:22:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/27 16:22:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/27 16:22:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


root
 |-- helpfulness: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- profile_name: string (nullable = true)
 |-- review: string (nullable = true)
 |-- score: double (nullable = true)
 |-- summary: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)



In [2]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, regexp_extract,sha1
from pyspark.sql.types import IntegerType, LongType
from numpy import array
import hashlib
import math
hex_to_bigint_udf = F.udf(lambda x: int(x, 16) % (10 ** 8), LongType())
ratings = ratings.withColumn('user_id', hex_to_bigint_udf(sha1(col('user_id').cast('string')))).withColumn('product_id', hex_to_bigint_udf(sha1(col('product_id').cast('string')))).select('user_id', 'product_id', col('score').cast('int'))
# ratings = ratings.withColumn("user_id", regexp_extract(col("user_id"), r"\d+", 0).cast(IntegerType()))
# ratings = ratings.withColumn("product_id", regexp_extract(col("product_id"), r"\d+", 0).cast(IntegerType()))
print(ratings.count())
# ratings=ratings.dropna()
#print(ratings.count())
ratings.show(1)
#ratings.show(1)


50000


+-------+----------+-----+
|user_id|product_id|score|
+-------+----------+-----+
|5460385|  51259877|    3|
+-------+----------+-----+
only showing top 1 row



In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

als = ALS( userCol = 'user_id', itemCol = 'product_id', ratingCol = 'score')

(train, valid) = ratings.randomSplit([8.0, 2.0])
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'score', predictionCol = 'prediction')
model = als.fit(train)
pred = model.transform(valid)

24/10/27 16:22:56 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/10/27 16:22:56 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [4]:
pred.show(3)

+-------+----------+-----+----------+
|user_id|product_id|score|prediction|
+-------+----------+-----+----------+
|1526715|  87115994|    5|       NaN|
| 423677|  89761594|    3| 1.2492025|
|1373002|  84390254|    5|       NaN|
+-------+----------+-----+----------+
only showing top 3 rows



In [5]:
print(evaluator.evaluate(pred.na.drop()))

1.966633948707933


In [6]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

param = ParamGridBuilder().addGrid(als.rank, [1, 5]).addGrid(als.maxIter, [10]).addGrid(als.regParam, [0.05]).addGrid(als.alpha, [1]).build()

In [7]:
type(param)

list

In [8]:
print(param)

[{Param(parent='ALS_0dfcea36732f', name='rank', doc='rank of the factorization'): 1, Param(parent='ALS_0dfcea36732f', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='ALS_0dfcea36732f', name='regParam', doc='regularization parameter (>= 0).'): 0.05, Param(parent='ALS_0dfcea36732f', name='alpha', doc='alpha for implicit preference'): 1.0}, {Param(parent='ALS_0dfcea36732f', name='rank', doc='rank of the factorization'): 5, Param(parent='ALS_0dfcea36732f', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='ALS_0dfcea36732f', name='regParam', doc='regularization parameter (>= 0).'): 0.05, Param(parent='ALS_0dfcea36732f', name='alpha', doc='alpha for implicit preference'): 1.0}]


In [9]:
cross = CrossValidator( estimator = als, estimatorParamMaps = param, evaluator = evaluator, numFolds = 2)
cmodel = cross.fit(train)
cpred = cmodel.transform(valid)

In [10]:
rmse = evaluator.evaluate(cpred.na.drop())
print(rmse)

4.693397532738114


In [12]:
model = cmodel.bestModel
print(model)

ALSModel: uid=ALS_0dfcea36732f, rank=1
